In [1]:
import os
import os.path as path
import gc
import re
import math
import json

In [2]:
import numpy as np
import pandas as pd

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
pd.set_option('display.max_row', 50)
pd.set_option('display.max_columns', 100)

In [6]:
# 디렉토리 기본 경로 설정
DIR_PATH = path.join('..', 'data')
DIR_SAVE_PATH = path.join('..', 'data', 'output')

print(DIR_PATH)
print(DIR_SAVE_PATH)

..\data
..\data\output


In [7]:
# bean_raw_data.csv
bean_read = pd.read_csv(path.join(DIR_PATH, 'bean_raw_data.csv'), low_memory=False)

print(bean_read.shape)
bean_read.head()

(52, 18)


,id,title,origin,region,rank,decaffeination,processing,coffeeing_note,roasting_point,other_note,store,aroma,flavor,acidity,sweetness,bitterness,body,balance
0,1,에티오피아 예가체프 G2,에티오피아,예가체프,G2,-,워시드,"꽃, 과일, 부드러운, 플로럴, 레몬, 허니",중배전,"핸드드립, 모카포트",커피창고,7,7,7,6,4,4,-
1,2,과테말라 안티구아,과테말라,안티구아,SHB,-,워시드,"스모크, 우아한, 중후한, 블랙커런트, 갈색설탕, 다크초코",강배전,"핸드드립, 모카포트",커피창고,6,6,3,7,7,7,-
2,3,에티오피아 코케허니 G1,에티오피아,코케,G1,-,펄프드내추럴,"베리, 체리, 허니",중배전,"핸드드립, 모카포트",커피창고,8,8,8,7,4,5,-
3,4,케냐 AA,케냐,키암부,AA,-,워시드,"자몽, 당밀, 카라멜",중배전,"핸드드립, 모카포트",커피창고,6,6,6,7,6,7,-
4,5,콜롬비아 수프리모,콜롬비아,콜롬비아,SUPREMO,-,워시드,"마일드, 적포도, 메이플시럽, 다크초코",중배전,"핸드드립, 모카포트",커피창고,7,7,5,6,6,6,-


In [8]:
bean_data = bean_read.copy()

In [9]:
# 카페인 결측치 채우기
bean_data.replace({'decaffeination':'-'}, 'F', inplace=True)
bean_data.head(1)

,id,title,origin,region,rank,decaffeination,processing,coffeeing_note,roasting_point,other_note,store,aroma,flavor,acidity,sweetness,bitterness,body,balance
0,1,에티오피아 예가체프 G2,에티오피아,예가체프,G2,F,워시드,"꽃, 과일, 부드러운, 플로럴, 레몬, 허니",중배전,"핸드드립, 모카포트",커피창고,7,7,7,6,4,4,-


In [10]:
bean_data.columns

Index(['id', 'title', 'origin', 'region', 'rank', 'decaffeination',
       'processing', 'coffeeing_note', 'roasting_point', 'other_note', 'store',
       'aroma', 'flavor', 'acidity', 'sweetness', 'bitterness', 'body',
       'balance'],
      dtype='object')

In [11]:
bean_data.dtypes

id                 int64
title             object
origin            object
region            object
rank              object
decaffeination    object
processing        object
coffeeing_note    object
roasting_point    object
other_note        object
store             object
aroma              int64
flavor             int64
acidity            int64
sweetness          int64
bitterness        object
body               int64
balance           object
dtype: object

In [12]:
# 숫자 결측치 채우기
# 단순하게 측정치의 중앙인 5로 결측치 보간
bean_data.replace({'aroma':'-', 'flavor':'-', 'acidity':'-', 'sweetness':'-', 'bitterness':'-', 'body':'-', 'balance':'-'}, '5', inplace=True)
bean_data = bean_data.astype({'aroma':np.int32, 'flavor':np.int32, 'acidity':np.int32, 'sweetness':np.int32, 'bitterness':np.int32, 'body':np.int32, 'balance':np.int32})
bean_data.head()

,id,title,origin,region,rank,decaffeination,processing,coffeeing_note,roasting_point,other_note,store,aroma,flavor,acidity,sweetness,bitterness,body,balance
0,1,에티오피아 예가체프 G2,에티오피아,예가체프,G2,F,워시드,"꽃, 과일, 부드러운, 플로럴, 레몬, 허니",중배전,"핸드드립, 모카포트",커피창고,7,7,7,6,4,4,5
1,2,과테말라 안티구아,과테말라,안티구아,SHB,F,워시드,"스모크, 우아한, 중후한, 블랙커런트, 갈색설탕, 다크초코",강배전,"핸드드립, 모카포트",커피창고,6,6,3,7,7,7,5
2,3,에티오피아 코케허니 G1,에티오피아,코케,G1,F,펄프드내추럴,"베리, 체리, 허니",중배전,"핸드드립, 모카포트",커피창고,8,8,8,7,4,5,5
3,4,케냐 AA,케냐,키암부,AA,F,워시드,"자몽, 당밀, 카라멜",중배전,"핸드드립, 모카포트",커피창고,6,6,6,7,6,7,5
4,5,콜롬비아 수프리모,콜롬비아,콜롬비아,SUPREMO,F,워시드,"마일드, 적포도, 메이플시럽, 다크초코",중배전,"핸드드립, 모카포트",커피창고,7,7,5,6,6,6,5


In [13]:
# # 리스트 컴프리헨션으로 coffeeing_note의 태그를 토큰화
# bean_data['coffeeing_note'] = bean_data['coffeeing_note'].apply(lambda x : [element.strip() for element in x.split(',')])
# bean_data.loc[1, 'coffeeing_note']

In [14]:
# 각 항목의 누적치를 확인하는 메소드
# 더 좋은 방법이 있을 듯?
df_bean_group = pd.DataFrame()
df_bean_group['id'] = pd.Series([x for x in range(10)])
df_bean_group[['aroma', 'flavor', 'acidity', 'sweetness', 'bitterness', 'body', 'balance']] = 0
df_bean_group.loc[bean_data[['aroma']].value_counts(dropna=False).reset_index().T.values[0], 'aroma'] = bean_data[['aroma']].value_counts(dropna=False).reset_index().T.values[1]
df_bean_group.loc[bean_data[['flavor']].value_counts(dropna=False).reset_index().T.values[0], 'flavor'] = bean_data[['flavor']].value_counts(dropna=False).reset_index().T.values[1]
df_bean_group.loc[bean_data[['acidity']].value_counts(dropna=False).reset_index().T.values[0], 'acidity'] = bean_data[['acidity']].value_counts(dropna=False).reset_index().T.values[1]
df_bean_group.loc[bean_data[['sweetness']].value_counts(dropna=False).reset_index().T.values[0], 'sweetness'] = bean_data[['sweetness']].value_counts(dropna=False).reset_index().T.values[1]
df_bean_group.loc[bean_data[['bitterness']].value_counts(dropna=False).reset_index().T.values[0], 'bitterness'] = bean_data[['bitterness']].value_counts(dropna=False).reset_index().T.values[1]
df_bean_group.loc[bean_data[['body']].value_counts(dropna=False).reset_index().T.values[0], 'body'] = bean_data[['body']].value_counts(dropna=False).reset_index().T.values[1]
df_bean_group.loc[bean_data[['balance']].value_counts(dropna=False).reset_index().T.values[0], 'balance'] = bean_data[['balance']].value_counts(dropna=False).reset_index().T.values[1]

df_bean_group.set_index('id', inplace=True)

In [15]:
df_bean_group

,aroma,flavor,acidity,sweetness,bitterness,body,balance
id,,,,,,,
0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0
2,0,0,3,0,1,0,0
3,0,0,2,0,0,0,0
4,5,5,17,0,5,4,0
5,1,1,7,0,28,2,32
6,18,18,14,24,13,19,7
7,8,8,3,10,4,11,0
8,19,19,6,18,1,15,13


In [16]:
# TF-IDF 벡터화
tfidf_vector = TfidfVectorizer()
tfidf_matrix = tfidf_vector.fit_transform(bean_data['coffeeing_note']).toarray()
tfidf_matrix_feature = tfidf_vector.get_feature_names_out()

In [17]:
print(tfidf_matrix_feature)
print(tfidf_matrix)

['가벼운' '갈색설탕' '감귤' '감초' '감칠맛' '강한' '개성전인' '건포도' '견과류' '고소한' '곡물' '과일'
 '과테말라' '균형있는' '균형잡힌' '깊은' '깔끔한' '다크초코' '달고나' '달콤판' '달콤한' '당밀' '딸기' '땅콩'
 '라임' '레드와인' '레몬' '마일드' '마일드한' '마카다미아' '맑은' '매끄러운' '메이플시럽' '묵직한' '밀크초코'
 '바닐라' '밝은' '베리' '보리' '복숭아' '복잡한' '부드러운' '브라질' '블랙커런트' '블루마운틴' '사과' '사탕수수'
 '산뜻한' '산토스' '살구' '세련된' '세이보리' '수프리모' '스모크' '스카치' '스카치캔디' '시트러스' '쌉쌀한'
 '아몬드' '아침' '안티구아' '에티오피아' '여운있는' '열대과일' '예가체프' '오렌지' '오크나무' '오트밀' '옥수수'
 '우아한' '자두' '자메이카' '자몽' '자연스러운' '적포도' '조청' '중후한' '진한' '청량한' '청포도' '체리'
 '초코' '카라멜' '카카오' '캔디' '코코넛밀크' '코코아' '콜롬비아' '쿠키' '크랜베리' '클래식' '토스트' '포도'
 '푸른사과' '풍부한' '플로럴' '피넛' '허니' '헤이즐넛' '현미' '호두파이' '호박고구마']
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.37572828 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.    

In [18]:
%%time
# tfidf_matrix 기반 유사도 측정
grade_cosine_sim = cosine_similarity(tfidf_matrix)

CPU times: total: 0 ns
Wall time: 1.99 ms


In [19]:
%%time
# aroma ~ balance 까지의 스테이터스에 따른 유사도 특정
grade_cosine_sim = cosine_similarity(bean_data[['aroma', 'flavor', 'acidity', 'sweetness', 'bitterness', 'body', 'balance']])

CPU times: total: 0 ns
Wall time: 1.83 ms


In [20]:
print(grade_cosine_sim.shape)
print(grade_cosine_sim.dtype)

grade_cosine_sim = grade_cosine_sim.astype(np.float16)
gc.collect()
print(grade_cosine_sim.dtype)

grade_cosine_sim

(52, 52)
float64
float16


array([[1.    , 0.9253, 0.9985, ..., 0.951 , 0.9316, 0.9316],
       [0.9253, 1.    , 0.9224, ..., 0.976 , 0.9756, 0.9756],
       [0.9985, 0.9224, 1.    , ..., 0.949 , 0.928 , 0.928 ],
       ...,
       [0.951 , 0.976 , 0.949 , ..., 1.    , 0.9917, 0.9917],
       [0.9316, 0.9756, 0.928 , ..., 0.9917, 1.    , 1.    ],
       [0.9316, 0.9756, 0.928 , ..., 0.9917, 1.    , 1.    ]],
      dtype=float16)

In [21]:
# 이름 뿐만 아니라 id로도 검색할 수 있도록 행과 열 중 하나를 title, 다른 하나를 id로 지정

df_grade_cosine_sim = pd.DataFrame(grade_cosine_sim, index = bean_data.id, columns = bean_data.title, dtype=np.float16)
print(df_grade_cosine_sim.shape)
df_grade_cosine_sim.head()

(52, 52)


title,에티오피아 예가체프 G2,과테말라 안티구아,에티오피아 코케허니 G1,케냐 AA,콜롬비아 수프리모,페루 엘 쿠엘로 워시드,브라질 세하도 파인컵 NY2,케냐 피베리 FAQ,파나마 라 에스메랄다 팔미라 에스테이트,브라질 옐로우 버번,브라질 이스페란사 내추럴,코스타리카 테라 벨라 하이브리드 마이크로랏,온두라스 엘 파라이소 파라이네마,에티오피아 시다모 봄베 워시드,과테말라 SHB 디카페인,콜롬비아 엑셀소 마운틴 워터 디카페인,인도 몬순 말라바르 AA,코스타리카 따라주 SHB,르완다 부산제,브라질 산토스 디카페인 마운틴 워터 디카페인,엘살바도르 놈브레 데 디오스 마이크로랏,인도네시아 만델링 G1,베트남 로부스타 G1,에티오피아 시다모 G2,인도 마이소르 너겟 엑스타르 볼듯,에티오피아 시다모 G2 스위스 워터 디카페인,콜롬비아 수프리모 EP SC 17 스위스 워터 디카페인,에티오피아 예가체프 G2,브라질 세하도 NY2 FC 17/18,과테말라 안티구아 SHB,콜롬비아 수프리모 후일라,케냐 AA FAQ,베트남 로부스타,브라질 산토스 NY2 FC,브라질 세하도 NY2 FC,에티오피아 시다모 G2,에티오피아 시다모 G4,에티오피아 예가체프 G2,에티오피아 예가체프 G4,과테말라 안티구아 SHB,탄자니아 킬리만자로 AA,콜롬비아 수프리모 메데인,코스타리카 따라주 SHB,파푸아뉴기니 아로나 AA,인도네시아 만델링 G1,온두라스 SHB,엘 살바도르 팬시,케냐 AA,콜롬비아 수프리모 마운틴 워터 디카페인,과테말라 안티구아 마운틴 워터 디카페인,에티오피아 예가체프 마운틴 워터 디카페인,브라질 산토스 마운틴 워터 디카페인
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.925293,0.998535,0.967773,0.976074,0.972656,0.965820,0.977051,0.961914,0.976074,0.986816,0.994141,0.988281,0.996094,0.952637,0.928711,0.943848,0.983398,0.990234,0.977051,0.985352,0.930664,0.924316,0.988281,0.949219,0.972168,0.970215,0.989746,0.926758,0.934570,0.970215,0.941406,0.976074,0.930664,0.874023,0.992676,0.988770,0.998047,0.988770,0.951172,0.951172,0.931641,0.929688,0.946289,0.931641,0.930664,0.949219,0.931641,0.931641,0.951172,0.931641,0.931641
2,0.925293,1.000000,0.922363,0.980957,0.982422,0.961426,0.980957,0.976074,0.992188,0.985352,0.954102,0.933594,0.958984,0.942383,0.992676,0.970215,0.982910,0.970215,0.951172,0.981445,0.969727,0.993164,0.996094,0.966797,0.994629,0.980469,0.973145,0.875000,0.984375,0.993652,0.973145,0.984375,0.969727,0.976562,0.954102,0.940430,0.959473,0.917969,0.959473,0.976074,0.976074,0.975586,0.934570,0.970703,0.975586,0.976562,0.968262,0.975586,0.975586,0.976074,0.975586,0.975586
3,0.998535,0.922363,1.000000,0.967285,0.973633,0.979004,0.963867,0.974121,0.960449,0.974121,0.990234,0.997559,0.990723,0.997559,0.950195,0.928711,0.943359,0.978516,0.993652,0.975586,0.986816,0.927734,0.919922,0.988281,0.947754,0.975098,0.971680,0.992188,0.926758,0.936035,0.971680,0.943359,0.974121,0.924805,0.861816,0.987793,0.986816,0.993652,0.986816,0.949219,0.949219,0.928223,0.920410,0.939453,0.928223,0.924805,0.940430,0.928223,0.928223,0.949219,0.928223,0.928223
4,0.967773,0.980957,0.967285,1.000000,0.990723,0.982422,0.998535,0.997559,0.994141,0.992676,0.977539,0.976074,0.987793,0.981445,0.996094,0.987305,0.967773,0.992676,0.987305,0.996582,0.993164,0.980957,0.969238,0.990723,0.992188,0.993652,0.987793,0.938477,0.961914,0.989258,0.987793,0.973633,0.992676,0.982422,0.933105,0.963379,0.979980,0.959961,0.979980,0.972656,0.972656,0.974121,0.962402,0.970703,0.974121,0.982422,0.969727,0.974121,0.974121,0.972656,0.974121,0.974121
5,0.976074,0.982422,0.973633,0.990723,1.000000,0.982910,0.988770,0.994629,0.997070,0.998535,0.984375,0.976074,0.991699,0.984863,0.990723,0.960449,0.988281,0.993164,0.985352,0.996582,0.994141,0.987305,0.981445,0.996094,0.992676,0.994629,0.995117,0.942383,0.978516,0.979980,0.995117,0.988281,0.983398,0.966309,0.926270,0.983887,0.993652,0.973633,0.993652,0.980469,0.980469,0.966309,0.941895,0.968750,0.966309,0.966309,0.974121,0.966309,0.966309,0.980469,0.966309,0.966309


In [22]:
df_grade_cosine_sim.loc[1].sort_values(ascending=False)[1:11].index

Index(['에티오피아 코케허니 G1', '에티오피아 예가체프 G2', '에티오피아 시다모 봄베 워시드',
       '코스타리카 테라 벨라 하이브리드 마이크로랏', '에티오피아 시다모 G2', '르완다 부산제', '에티오피아 예가체프 G2',
       '에티오피아 시다모 G4', '에티오피아 예가체프 G4', '에티오피아 시다모 G2'],
      dtype='object', name='title')

In [23]:
df_grade_cosine_sim.index.get_indexer([1])

array([0], dtype=int64)

In [24]:
df_grade_cosine_sim.iloc[:, df_grade_cosine_sim.index.get_indexer([1])].columns[0]

'에티오피아 예가체프 G2'

In [25]:
df_grade_cosine_sim.iloc[:, df_grade_cosine_sim.index.get_indexer([1])].sort_values(by=df_grade_cosine_sim.iloc[:, df_grade_cosine_sim.index.get_indexer([1])].columns[0], ascending=False)[1:11].index

Int64Index([3, 38, 14, 12, 36, 19, 28, 37, 39, 24], dtype='int64', name='id')

In [26]:
# id 기반 추천 알고리즘
def recommendations_by_id(target_id, matrix, items, k=10):
    try:
        target_idx =  matrix.index.get_indexer([target_id])
        recom_idx = matrix.iloc[:, target_idx].sort_values(by= matrix.iloc[:, target_idx].columns[0], ascending=False)[1:11].index

        # 반환한 인덱스 값은 1부터 시작하나, 실제 iloc로 접근하는 인덱스 값은 0부터 시작하므로 이를 보정해야함
        recom_idx = recom_idx-1
        recom_id = items.iloc[recom_idx, :].id.values
        recom_title = items.iloc[recom_idx, :].title.values

        target_id_list = np.full(len(range(k)), target_id)
        target_title_list = np.full(len(range(k)), items[items.id == target_id].title.values)
        
    except:
        print(recom_idx)
        print(recom_id, recom_title)
        print(target_id_list, target_title_list)
    
    d = {
        'target_id': target_id_list,
        'target_title': target_title_list,
        'recom_id'    : recom_id,
        'recom_title' : recom_title,
    }
    
    return pd.DataFrame(d)

In [27]:
recommendations_by_id(42, df_grade_cosine_sim, bean_data)

,target_id,target_title,recom_id,recom_title
0,42,콜롬비아 수프리모 메데인,49,콜롬비아 수프리모 마운틴 워터 디카페인
1,42,콜롬비아 수프리모 메데인,51,에티오피아 예가체프 마운틴 워터 디카페인
2,42,콜롬비아 수프리모 메데인,42,콜롬비아 수프리모 메데인
3,42,콜롬비아 수프리모 메데인,45,인도네시아 만델링 G1
4,42,콜롬비아 수프리모 메데인,48,케냐 AA
5,42,콜롬비아 수프리모 메데인,44,파푸아뉴기니 아로나 AA
6,42,콜롬비아 수프리모 메데인,41,탄자니아 킬리만자로 AA
7,42,콜롬비아 수프리모 메데인,34,브라질 산토스 NY2 FC
8,42,콜롬비아 수프리모 메데인,46,온두라스 SHB
9,42,콜롬비아 수프리모 메데인,40,과테말라 안티구아 SHB


In [28]:
recommendations_by_id(31, df_grade_cosine_sim, bean_data)

,target_id,target_title,recom_id,recom_title
0,31,콜롬비아 수프리모 후일라,31,콜롬비아 수프리모 후일라
1,31,콜롬비아 수프리모 후일라,24,에티오피아 시다모 G2
2,31,콜롬비아 수프리모 후일라,5,콜롬비아 수프리모
3,31,콜롬비아 수프리모 후일라,26,에티오피아 시다모 G2 스위스 워터 디카페인
4,31,콜롬비아 수프리모 후일라,13,온두라스 엘 파라이소 파라이네마
5,31,콜롬비아 수프리모 후일라,20,브라질 산토스 디카페인 마운틴 워터 디카페인
6,31,콜롬비아 수프리모 후일라,21,엘살바도르 놈브레 데 디오스 마이크로랏
7,31,콜롬비아 수프리모 후일라,9,파나마 라 에스메랄다 팔미라 에스테이트
8,31,콜롬비아 수프리모 후일라,37,에티오피아 시다모 G4
9,31,콜롬비아 수프리모 후일라,39,에티오피아 예가체프 G4


In [29]:
# id 기반 추천 알고리즘
def recommendation_list_by_id(target_id, matrix, items, k=10):
    try:
        target_idx =  matrix.index.get_indexer([target_id])
        recom_idx = matrix.iloc[:, target_idx].sort_values(by= matrix.iloc[:, target_idx].columns[0], ascending=False)[1:11].index
        
        # 반환한 인덱스 값은 1부터 시작하나, 실제 iloc로 접근하는 인덱스 값은 0부터 시작하므로 이를 보정해야함
        recom_idx = recom_idx-1
        recom_id = items.iloc[recom_idx, :].id.values
        recom_title = items.iloc[recom_idx, :].title.values
        
    except:
        print(recom_idx)
        print(recom_id, recom_title)
    
    recom_list = [dict(id = id, title = title) for id, title in zip(recom_id, recom_title)]
    
    return recom_list

In [30]:
recommendation_list_by_id(42, df_grade_cosine_sim, bean_data)

[{'id': 49, 'title': '콜롬비아 수프리모 마운틴 워터 디카페인'},
 {'id': 51, 'title': '에티오피아 예가체프 마운틴 워터 디카페인'},
 {'id': 42, 'title': '콜롬비아 수프리모 메데인'},
 {'id': 45, 'title': '인도네시아 만델링 G1'},
 {'id': 48, 'title': '케냐 AA'},
 {'id': 44, 'title': '파푸아뉴기니 아로나 AA'},
 {'id': 41, 'title': '탄자니아 킬리만자로 AA'},
 {'id': 34, 'title': '브라질 산토스 NY2 FC'},
 {'id': 46, 'title': '온두라스 SHB'},
 {'id': 40, 'title': '과테말라 안티구아 SHB'}]

In [31]:
print(df_grade_cosine_sim.shape)

(52, 52)


In [32]:
print(bean_data.shape)

(52, 18)


In [33]:
# 유사도 기준으로 추천 원두의 상위 5개를 출력
bean_recom = bean_data.copy()[['id', 'title']]
bean_recom['recommendation'] = bean_recom.apply(lambda x: recommendation_list_by_id(x.id, df_grade_cosine_sim, bean_data, k=5), axis=1)
bean_recom

,id,title,recommendation
0,1,에티오피아 예가체프 G2,"[{'id': 3, 'title': '에티오피아 코케허니 G1'}, {'id': 3..."
1,2,과테말라 안티구아,"[{'id': 23, 'title': '베트남 로부스타 G1'}, {'id': 25..."
2,3,에티오피아 코케허니 G1,"[{'id': 1, 'title': '에티오피아 예가체프 G2'}, {'id': 1..."
3,4,케냐 AA,"[{'id': 7, 'title': '브라질 세하도 파인컵 NY2'}, {'id':..."
4,5,콜롬비아 수프리모,"[{'id': 10, 'title': '브라질 옐로우 버번'}, {'id': 9, ..."
...,...,...,...
47,48,케냐 AA,"[{'id': 49, 'title': '콜롬비아 수프리모 마운틴 워터 디카페인'},..."
48,49,콜롬비아 수프리모 마운틴 워터 디카페인,"[{'id': 49, 'title': '콜롬비아 수프리모 마운틴 워터 디카페인'},..."
49,50,과테말라 안티구아 마운틴 워터 디카페인,"[{'id': 50, 'title': '과테말라 안티구아 마운틴 워터 디카페인'},..."
50,51,에티오피아 예가체프 마운틴 워터 디카페인,"[{'id': 49, 'title': '콜롬비아 수프리모 마운틴 워터 디카페인'},..."


In [34]:
# 파일 저장
os.makedirs(DIR_SAVE_PATH, exist_ok=True)
# bean_recom.to_csv(path.join(DIR_SAVE_PATH, 'bean_cbf_recom.csv'), sep=',')
bean_recom.to_csv(path.join(DIR_SAVE_PATH, 'bean_cbf_recom.csv'), sep=',', index=False)